<a href="https://colab.research.google.com/github/Pravanop/ml4Sci_eval/blob/main/Task_3_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyarrow

In [ ]:
!cd '/content/drive/MyDrive/GSOC/Application/ML4Sci/Task 3/dataset'

In [ ]:
!unzip '/content/drive/MyDrive/GSOC/Application/ML4Sci/Task 3/download (1).zip' 

Archive:  /content/drive/MyDrive/GSOC/Application/ML4Sci/Task 3/download (1).zip
 extracting: E2E_Regression.parquet.1  
 extracting: E2E_Regression.parquet.10  
 extracting: E2E_Regression.parquet.11  
 extracting: E2E_Regression.parquet.12  
 extracting: E2E_Regression.parquet.14  
 extracting: E2E_Regression.parquet.15  
 extracting: E2E_Regression.parquet.16  
 extracting: E2E_Regression.parquet.17  
 extracting: E2E_Regression.parquet.18  
 extracting: E2E_Regression.parquet.19  
 extracting: E2E_Regression.parquet.2  
 extracting: E2E_Regression.parquet.3  
 extracting: E2E_Regression.parquet.4  
 extracting: E2E_Regression.parquet.5  
 extracting: E2E_Regression.parquet.6  
 extracting: E2E_Regression.parquet.7  
 extracting: E2E_Regression.parquet.8  
 extracting: E2E_Regression.parquet.9  


In [ ]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy

In [ ]:
class jetDataset(Dataset):

    def __init__(self, path):
      self.table = pq.ParquetFile(path)

    def __len__(self):
        return self.table.num_row_groups

    def __getitem__(self, idx):
        row = self.table.read_row_group(idx)
        jet = row.column('X_jet').to_numpy()
        label = row.column('y').to_numpy()
        three_array = []
        for j,m in enumerate(jet[0]):
          arr = []
          for i,k in enumerate(jet[0][j]):
            if i == 0:
              arr = k
              arr = arr.reshape(arr.shape[0],1)
            else:
              k = k.reshape(k.shape[0],1)
              arr = np.concatenate((arr,k),axis=0)
          if j == 0 :
            three_array = arr
          else:
            three_array = np.concatenate((three_array,arr),axis=0)
        
        three_array_reshaped = three_array.reshape(125,125,4)
        am = row.column('am').to_numpy()[0]
        am_arr = np.ones((125,125))*am
        image = np.dstack((three_array_reshaped[:,:,0],three_array_reshaped[:,:,-1],am_arr)) #using pt,ecal and am as features for the image
        x = torch.tensor(image)
        label = label[0]
        
        sample = (x, label)
        return sample

In [ ]:
#had to manually extract due to discrepancy in colab
data1 = jetDataset('/content/E2E_Regression.parquet.1')
data2 = jetDataset('/content/E2E_Regression.parquet.2')
data3 = jetDataset('/content/E2E_Regression.parquet.3')
data4 = jetDataset('/content/E2E_Regression.parquet.4')
data5 = jetDataset('/content/E2E_Regression.parquet.5')
data6 = jetDataset('/content/E2E_Regression.parquet.6')
data7 = jetDataset('/content/E2E_Regression.parquet.7')
data8 = jetDataset('/content/E2E_Regression.parquet.8')
data9 = jetDataset('/content/E2E_Regression.parquet.9')
data10 = jetDataset('/content/E2E_Regression.parquet.10')
data11 = jetDataset('/content/E2E_Regression.parquet.11')
data12 = jetDataset('/content/E2E_Regression.parquet.12')
data14 = jetDataset('/content/E2E_Regression.parquet.14')
data15 = jetDataset('/content/E2E_Regression.parquet.15')
data16 = jetDataset('/content/E2E_Regression.parquet.16')
data17 = jetDataset('/content/E2E_Regression.parquet.17')
data18 = jetDataset('/content/E2E_Regression.parquet.18')
data19 = jetDataset('/content/E2E_Regression.parquet.19')

In [ ]:
sample = data17[0][1]
type(sample)

numpy.float64

In [ ]:
all_datasets = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12,data14,data15,data16,data17,data18,data19]
final_dataset = torch.utils.data.ConcatDataset(all_datasets)

In [ ]:
train_size = int(0.8 * len(final_dataset))
test_size = len(final_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(final_dataset, [train_size, test_size])

In [ ]:
datasets = {'train':train_dataset,'val':test_dataset}
dataloaders = {x: DataLoader(datasets[x], batch_size= 1024, shuffle=True, num_workers=0) for x in ['train','val']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(dataset_sizes)

{'train': 92284, 'val': 23071}


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [ ]:
res_mod = models.resnet34(pretrained=True)
num_ftrs = res_mod.fc.in_features
res_mod.fc = nn.Linear(num_ftrs, 1)

In [ ]:
res_mod = res_mod.to(device)
criterion = nn.MSELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(res_mod.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            print('Iterating through data...')

            for inputs, labels in dataloaders[phase]:
                torch.cuda.empty_cache()
                inputs = inputs.permute(0, 3, 1, 2)
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Make a copy of the model if the accuracy on the validation set has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
base_model = train_model(res_mod, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

Epoch 0/0
----------
Iterating through data...


RuntimeError: ignored